In [6]:
import pprint 
import psycopg2 
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT 
from psycopg2.extensions import AsIs 
import os

def pg_con(): 
    con = psycopg2.connect(
            dbname='postgres', 
            user='postgres', 
            host='127.0.0.1', 
            port= '5432', 
            password='admin'
    )
    return(con) 

In [7]:
pg_con()

OperationalError: could not connect to server: Connection refused
	Is the server running on host "127.0.0.1" and accepting
	TCP/IP connections on port 5432?
